In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from keras import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Dropout, Normalization
from keras.models import Model
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [10]:
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print("NUM GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

NUM GPUs Available:  1


In [11]:
model = MobileNetV2(include_top=False, input_shape=(32, 32,3))

In [12]:
train_path = "data/train"
test_path = "data/test"
labels = {"akiec":0, "bcc":1, "bkl":2, "df":3, "mel":4, "nv":5, "vasc":6}
classes = list(labels.keys())
size = (32,32)

In [13]:
model_generator = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range = 40,
                                        width_shift_range = 0.2, height_shift_range = 0.2, rescale = 1./255,
                                        shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True,
                                        validation_split=.2)

train_batches = model_generator.flow_from_directory(train_path,target_size=size,classes=classes,
                                                batch_size=10, shuffle=True, subset="training")
validation_batches = model_generator.flow_from_directory(train_path,size,classes=classes,
                                                batch_size=20, shuffle=True, subset="validation")
test_batches = ImageDataGenerator(preprocessing_function=preprocess_input)\
    .flow_from_directory(test_path,size,classes=classes, batch_size=10, shuffle=False)

Found 6413 images belonging to 7 classes.
Found 1599 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.


In [14]:
flat1 = Flatten()(model.layers[-1].output)
norm = Normalization()(flat1)
class1 = Dense(512, activation="relu")(norm)
class2 = Dropout(rate=.3)(class1)
class3 = Dense(128, activation="relu")(class2)
class4 = Dense(32, activation="relu")(class3)
output = Dense(7, activation="softmax")(class4)
model = Model(inputs=model.inputs, outputs=output)

In [15]:
learning_rate = 0.006
optimizer = Adam(learning_rate)
loss = "categorical_crossentropy"
metrics = "accuracy"

model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=metrics)

In [16]:
model.fit(train_batches, validation_data= validation_batches,
          epochs=10, shuffle=True, verbose=2)

Epoch 1/10
642/642 - 48s - loss: 1.3267 - accuracy: 0.6601 - val_loss: 1.2791 - val_accuracy: 0.6704
Epoch 2/10


KeyboardInterrupt: 

In [ ]:
model.save("deployment/models/best_model.h5")